<a href="https://colab.research.google.com/github/AlienMedoff/4.01/blob/main/801.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, json, math, requests
from datetime import datetime, timedelta, timezone

# --- КОНФИГУРАЦИЯ ШКОЛА ---
CONFIG = {
    'API_KEY_SPORTS': 'd5f8091f602cb270431726b11d7100cc',
    'POSSIBLE_PATHS': [
        '/content/drive/MyDrive/ШКОЛА/patterns.json',
        '/content/drive/My Drive/ШКОЛА/patterns.json'
    ]
}

def mathematician_check(h_o, a_o, target_score, math_db):
    """Математик: поиск по историческому радиусу 0.20"""
    sims = [p['score'] for p in math_db if math.sqrt((p['h']-h_o)**2 + (p['a']-a_o)**2) < 0.20]
    if not sims: return 0, 0
    count = sims.count(target_score)
    prob = (count / len(sims)) * 100
    return prob, len(sims)

def physicist_dictator(h_o, a_o, dens):
    """Физик: Диктатура сценариев (Калибровка PKM 5.0)"""
    is_mirrored = abs(h_o - a_o) < 0.5
    fav_home = h_o < a_o

    if dens > 0.55:
        label = "🔥 ЖИР: ТОТАЛЬНЫЙ ВЫНОС (ТБ 3.5)"
        if is_mirrored: scores = ['2:2', '3:2', '2:3']
        else: scores = ['3:1', '4:1', '4:0'] if fav_home else ['1:3', '1:4', '0:4']
    elif is_mirrored and dens > 0.20:
        label = "💎 БЕТОН: РЕЗУЛЬТАТИВНАЯ БИТВА (ОЗ)"
        scores = ['1:1', '2:1', '1:2', '2:2']
    else:
        label = "🔍 СТАНДАРТ: КЛАСС"
        scores = ['2:0', '3:0', '2:1'] if fav_home else ['0:2', '0:3', '1:2']
    return scores, label

def get_dens_3_5(odds_list):
    """Калибровка парсинга: Ищем именно Over 3.5"""
    for bet in odds_list:
        if "Over/Under" in bet['name']:
            for val in bet['values']:
                if val['value'] == 'Over 3.5':
                    return round(1 / float(val['odd']), 2)
    for bet in odds_list:
        if "Over/Under" in bet['name']:
            for val in bet['values']:
                if val['value'] == 'Over 2.5':
                    return round((1 / float(val['odd'])) * 0.7, 2)
    return 0.30

def run_pkm_5_0():
    # Ищем базу ШКОЛА по всем возможным путям
    path_db = None
    for p in CONFIG['POSSIBLE_PATHS']:
        if os.path.exists(p):
            path_db = p
            break

    if not path_db:
        print("❌ БАЗА НЕ НАЙДЕНА!")
        print(f"Пробовал искать тут: {CONFIG['POSSIBLE_PATHS']}")
        print("Убедись, что Google Drive примонтирован: drive.mount('/content/drive')")
        return

    with open(path_db, 'r') as f:
        math_db = json.load(f)

    now = datetime.now(timezone.utc)
    headers = {'x-apisports-key': CONFIG['API_KEY_SPORTS']}
    print(f"--- 🛰 PKM 5.0: FULL CALIBRATION | {now.strftime('%d/%m %H:%M')} ---")

    try:
        res = requests.get(f"https://v3.football.api-sports.io/fixtures?date={now.strftime('%Y-%m-%d')}", headers=headers).json()
    except:
        return print("❌ Ошибка сети/API")

    for f in res.get('response', []):
        f_dt = datetime.fromisoformat(f['fixture']['date'].replace('Z', '+00:00'))
        if now <= f_dt <= now + timedelta(hours=12):
            try:
                fid = f['fixture']['id']
                o_res = requests.get(f"https://v3.football.api-sports.io/odds?fixture={fid}", headers=headers).json()
                if not o_res['response']: continue
                bookie = o_res['response'][0]['bookmakers'][0]

                odds = {}
                for b in bookie['bets']:
                    if b['name'] in ['Match Winner', 'Full Time Result']:
                        odds = {v['value']: float(v['odd']) for v in b['values']}

                h_o, a_o = odds.get('Home', odds.get('1')), odds.get('Away', odds.get('2'))
                if not h_o or not a_o: continue
                dens = get_dens_3_5(bookie['bets'])

                suggested_scores, label = physicist_dictator(h_o, a_o, dens)
                final_results = []
                for s in suggested_scores:
                    prob, matches = mathematician_check(h_o, a_o, s, math_db)
                    final_results.append({'score': s, 'prob': prob, 'm': matches})

                final_results.sort(key=lambda x: x['prob'], reverse=True)
                top_choice = final_results[0]

                print(f"\n⚽ {f['league']['name']} | {f['teams']['home']['name']} - {f['teams']['away']['name']}")
                print(f"📊 Линия: [{h_o} - {a_o}] | Dens(3.5): {dens}")
                print(f"📢 СЦЕНАРИЙ: {label}")
                for r in final_results:
                    print(f"   - Вариант {r['score']}: Ист. Вероятность {r['prob']:.1f}% ({r['m']} матчей)")

                if top_choice['prob'] < 5 and dens < 0.50:
                    print("⚠️ СТАТУС: СЛАБЫЙ СИГНАЛ (SKIP)")
                else:
                    print(f"🎯 ИТОГОВЫЙ ВЫБОР: {top_choice['score']}")
                print("-" * 55)
            except: continue

if __name__ == "__main__":
    run_pkm_5_0()

--- 🛰 PKM 5.0: FULL CALIBRATION | 08/01 04:51 ---

⚽ Friendlies Clubs | Lausanne - Neuchatel Xamax FC
📊 Линия: [1.44 - 5.0] | Dens(3.5): 0.1
📢 СЦЕНАРИЙ: 🔍 СТАНДАРТ: КЛАСС
   - Вариант 2:1: Ист. Вероятность 12.7% (55 матчей)
   - Вариант 2:0: Ист. Вероятность 10.9% (55 матчей)
   - Вариант 3:0: Ист. Вероятность 10.9% (55 матчей)
🎯 ИТОГОВЫЙ ВЫБОР: 2:1
-------------------------------------------------------

⚽ Premier League | Suhul Shire - Negelle Arsi
📊 Линия: [3.6 - 2.1] | Dens(3.5): 0.3
📢 СЦЕНАРИЙ: 🔍 СТАНДАРТ: КЛАСС
   - Вариант 1:2: Ист. Вероятность 7.4% (406 матчей)
   - Вариант 0:2: Ист. Вероятность 6.7% (406 матчей)
   - Вариант 0:3: Ист. Вероятность 4.2% (406 матчей)
🎯 ИТОГОВЫЙ ВЫБОР: 1:2
-------------------------------------------------------

⚽ Friendlies Clubs | Muscelul Câmpulung Elite - Unirea Slobozia
📊 Линия: [41.0 - 1.04] | Dens(3.5): 0.28
📢 СЦЕНАРИЙ: 🔍 СТАНДАРТ: КЛАСС
   - Вариант 0:2: Ист. Вероятность 0.0% (0 матчей)
   - Вариант 0:3: Ист. Вероятность 0.0% (0 матчей)
  

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
import os, json, math, requests
from datetime import datetime, timedelta, timezone

# --- АВТОКОННЕКТОР С ДИСКОМ ---
try:
    from google.colab import drive
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive', force_remount=True)
except ImportError:
    print("⚠️ Запуск не в Colab? Убедись, что пути к папке ШКОЛА верны.")

# --- КОНФИГУРАЦИЯ ШКОЛА ---
CONFIG = {
    'API_KEY_SPORTS': 'd5f8091f602cb270431726b11d7100cc',
    'POSSIBLE_PATHS': [
        '/content/drive/MyDrive/ШКОЛА/patterns.json',
        '/content/drive/My Drive/ШКОЛА/patterns.json'
    ]
}

def mathematician_check(h_o, a_o, target_score, math_db):
    """Математик: Динамический радиус для фаворитов"""
    radius = 0.30 if min(h_o, a_o) < 1.60 else 0.20
    sims = [p['score'] for p in math_db if math.sqrt((p['h']-h_o)**2 + (p['a']-a_o)**2) < radius]
    if not sims: return 0, 0
    count = sims.count(target_score)
    prob = (count / len(sims)) * 100
    return prob, len(sims)

def physicist_dictator(h_o, a_o, dens):
    """Физик: Диктатура сценариев + Анти-Ловушка"""
    is_mirrored = abs(h_o - a_o) < 0.5
    fav_home = h_o < a_o
    min_odd = min(h_o, a_o)

    # ЛОВУШКА: Сверхфаворит при низкой плотности (как City вчера)
    if min_odd < 1.60 and dens < 0.45 and not is_mirrored:
        label = "⚠️ ЛОВУШКА: РИСК НИЧЬЕЙ (X2 / +1.5)"
        if fav_home: scores = ['1:1', '2:1', '1:0', '0:0']
        else: scores = ['1:1', '1:2', '0:1', '0:0']

    # ТОТАЛЬНЫЙ ЖИР
    elif dens > 0.55:
        label = "🔥 ЖИР: ТОТАЛЬНЫЙ ВЫНОС (ТБ 3.5)"
        if is_mirrored: scores = ['2:2', '3:2', '2:3']
        else: scores = ['3:1', '4:1', '4:0'] if fav_home else ['1:3', '1:4', '0:4']

    # ЗЕРКАЛЬНЫЙ БЕТОН (Твое правило для ШКОЛА)
    elif is_mirrored and dens > 0.20:
        label = "💎 БЕТОН: РЕЗУЛЬТАТИВНАЯ БИТВА (ОЗ)"
        scores = ['1:1', '2:1', '1:2', '2:2']

    else:
        label = "🔍 СТАНДАРТ: КЛАСС"
        if fav_home: scores = ['2:0', '3:0', '2:1', '1:1']
        else: scores = ['0:2', '0:3', '1:2', '1:1']

    return scores, label

def get_dens_3_5(odds_list):
    """Парсинг ТБ 3.5"""
    for bet in odds_list:
        if "Over/Under" in bet['name']:
            for val in bet['values']:
                if val['value'] == 'Over 3.5':
                    return round(1 / float(val['odd']), 2)
    for bet in odds_list:
        if "Over/Under" in bet['name']:
            for val in bet['values']:
                if val['value'] == 'Over 2.5':
                    return round((1 / float(val['odd'])) * 0.7, 2)
    return 0.35

def run_pkm_5_0():
    path_db = None
    for p in CONFIG['POSSIBLE_PATHS']:
        if os.path.exists(p): path_db = p; break

    if not path_db:
        print("❌ БАЗА ВСЕ ЕЩЕ НЕ НАЙДЕНА!")
        print("Проверь, чтобы папка называлась именно ШКОЛА (заглавными) иpatterns.json был внутри.")
        return

    with open(path_db, 'r') as f: math_db = json.load(f)

    now = datetime.now(timezone.utc)
    headers = {'x-apisports-key': CONFIG['API_KEY_SPORTS']}
    print(f"--- 🛰 PKM 5.0: FULL CALIBRATION | {now.strftime('%d/%m %H:%M')} ---")

    res = requests.get(f"https://v3.football.api-sports.io/fixtures?date={now.strftime('%Y-%m-%d')}", headers=headers).json()

    for f in res.get('response', []):
        f_dt = datetime.fromisoformat(f['fixture']['date'].replace('Z', '+00:00'))
        if now <= f_dt <= now + timedelta(hours=12):
            try:
                fid = f['fixture']['id']
                o_res = requests.get(f"https://v3.football.api-sports.io/odds?fixture={fid}", headers=headers).json()
                if not o_res['response']: continue
                bookie = o_res['response'][0]['bookmakers'][0]

                odds = {}
                for b in bookie['bets']:
                    if b['name'] in ['Match Winner', 'Full Time Result']:
                        odds = {v['value']: float(v['odd']) for v in b['values']}

                h_o, a_o = odds.get('Home', odds.get('1')), odds.get('Away', odds.get('2'))
                if not h_o or not a_o: continue
                dens = get_dens_3_5(bookie['bets'])

                suggested_scores, label = physicist_dictator(h_o, a_o, dens)
                final_results = []
                for s in suggested_scores:
                    prob, matches = mathematician_check(h_o, a_o, s, math_db)
                    if "ЛОВУШКА" in label and s == '1:1': prob += 10 # Бонус за подозрительную ничью
                    final_results.append({'score': s, 'prob': prob, 'm': matches})

                final_results.sort(key=lambda x: x['prob'], reverse=True)
                top_choice = final_results[0]

                print(f"\n⚽ {f['league']['name']} | {f['teams']['home']['name']} - {f['teams']['away']['name']}")
                print(f"📊 Линия: [{h_o} - {a_o}] | Dens(3.5): {dens}")
                print(f"📢 ВЕРДИКТ: {label}")
                for r in final_results:
                    print(f"   - {r['score']}: {r['prob']:.1f}% ({r['m']} матчей)")

                print(f"🎯 ИТОГОВЫЙ ВЫБОР: {top_choice['score']}")
                print("-" * 55)
            except: continue

if __name__ == "__main__":
    run_pkm_5_0()

Mounted at /content/drive
--- 🛰 PKM 5.0: FULL CALIBRATION | 11/01 16:25 ---

⚽ Premiership | Aberdeen - Rangers
📊 Линия: [4.35 - 1.78] | Dens(3.5): 0.32
📢 ВЕРДИКТ: 🔍 СТАНДАРТ: КЛАСС
   - 1:2: 11.0% (346 матчей)
   - 1:1: 11.0% (346 матчей)
   - 0:2: 9.0% (346 матчей)
   - 0:3: 4.0% (346 матчей)
🎯 ИТОГОВЫЙ ВЫБОР: 1:2
-------------------------------------------------------

⚽ Bundesliga | Bayern München - VfL Wolfsburg
📊 Линия: [1.12 - 18.5] | Dens(3.5): 0.65
📢 ВЕРДИКТ: 🔥 ЖИР: ТОТАЛЬНЫЙ ВЫНОС (ТБ 3.5)
   - 3:1: 0.0% (0 матчей)
   - 4:1: 0.0% (0 матчей)
   - 4:0: 0.0% (0 матчей)
🎯 ИТОГОВЫЙ ВЫБОР: 3:1
-------------------------------------------------------

⚽ Segunda División RFEF - Group 2 | Tudelano - Real Unión
📊 Линия: [2.65 - 2.55] | Dens(3.5): 0.32
📢 ВЕРДИКТ: 💎 БЕТОН: РЕЗУЛЬТАТИВНАЯ БИТВА (ОЗ)
   - 1:1: 16.3% (688 матчей)
   - 2:1: 8.7% (688 матчей)
   - 1:2: 8.7% (688 матчей)
   - 2:2: 5.2% (688 матчей)
🎯 ИТОГОВЫЙ ВЫБОР: 1:1
-------------------------------------------------------

⚽

In [ ]:
import os, json, math, requests
from datetime import datetime, timedelta, timezone

# --- АВТОКОННЕКТОР ---
try:
    from google.colab import drive
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive', force_remount=True)
except: pass

CONFIG = {
    'API_KEY_SPORTS': 'd5f8091f602cb270431726b11d7100cc',
    'POSSIBLE_PATHS': ['/content/drive/MyDrive/ШКОЛА/patterns.json', '/content/drive/My Drive/ШКОЛА/patterns.json']
}

def mathematician_check(h_o, a_o, target_score, math_db):
    radius = 0.30
    sims = [p['score'] for p in math_db if math.sqrt((p['h']-h_o)**2 + (p['a']-a_o)**2) < radius]
    if not sims: return 0.0, 0
    count = sims.count(target_score)
    return (count / len(sims)) * 100, len(sims)

def get_real_dens(bets):
    """Строгий парсинг линии без фиксов"""
    for b in bets:
        if "Over/Under" in b['name'] or "Total Goals" in b['name']:
            for v in b['values']:
                if v['value'] == 'Over 3.5':
                    return round(1 / float(v['odd']), 4)
    return None # Если в линии нет 3.5, возвращаем None для скипа

def physicist_dictator(h_o, a_o, dens):
    is_mirrored = abs(h_o - a_o) < 0.5
    fav_home = h_o < a_o
    min_odd = min(h_o, a_o)
    diff = abs(h_o - a_o)

    if min_odd < 1.60 and dens < 0.45 and not is_mirrored:
        label, scores = "⚠️ ЛОВУШКА", (['1:1', '2:1', '1:0', '0:0'] if fav_home else ['1:1', '1:2', '0:1', '0:0'])
    elif dens > 0.55:
        label, scores = "🔥 ЖИР", (['3:1', '4:1', '3:0'] if fav_home else ['1:3', '1:4', '0:3'])
    elif is_mirrored and dens > 0.20:
        label, scores = "💎 БЕТОН (ОЗ)", ['1:1', '2:1', '1:2', '2:2']
    else:
        label, scores = "🔍 СТАНДАРТ", (['2:0', '3:0', '2:1'] if fav_home else ['0:2', '0:3', '1:2'])
        if not (min_odd < 2.0 or diff > 2.0): scores.append('1:1')

    return scores, label

def run_pkm_5_0():
    path_db = next((p for p in CONFIG['POSSIBLE_PATHS'] if os.path.exists(p)), None)
    if not path_db: return print("❌ База ШКОЛА не найдена")

    with open(path_db, 'r') as f: math_db = json.load(f)
    now = datetime.now(timezone.utc)
    headers = {'x-apisports-key': CONFIG['API_KEY_SPORTS']}
    print(f"--- 🛰 PKM 5.0 STRICT | {now.strftime('%d/%m %H:%M')} ---")

    res = requests.get(f"https://v3.football.api-sports.io/fixtures?date={now.strftime('%Y-%m-%d')}", headers=headers).json()

    for f in res.get('response', []):
        f_dt = datetime.fromisoformat(f['fixture']['date'].replace('Z', '+00:00'))
        if now <= f_dt <= now + timedelta(hours=10):
            try:
                fid = f['fixture']['id']
                o_res = requests.get(f"https://v3.football.api-sports.io/odds?fixture={fid}", headers=headers).json()
                if not o_res['response']: continue

                bookie = o_res['response'][0]['bookmakers'][0]
                odds = {v['value']: float(v['odd']) for b in bookie['bets'] if b['name'] in ['Match Winner', 'Full Time Result'] for v in b['values']}
                h_o, a_o = odds.get('Home', odds.get('1')), odds.get('Away', odds.get('2'))
                if not h_o or not a_o: continue

                dens = get_real_dens(bookie['bets'])
                if dens is None: continue # СКИП: Нет данных по ТБ 3.5 в линии

                suggested_scores, label = physicist_dictator(h_o, a_o, dens)
                weighted_results = []

                for s in suggested_scores:
                    prob, matches = mathematician_check(h_o, a_o, s, math_db)
                    if prob == 0: continue # Исключаем счета с нулевой историей

                    # ПРИМЕНЕНИЕ ВЕСОВ
                    if dens > 0.40 and sum(int(x) for x in s.split(':')) > 2: prob *= 1.15
                    if min(h_o, a_o) < 1.70 and (':0' in s or '0:' in s): prob *= 1.10
                    if "ЛОВУШКА" in label and s == '1:1': prob *= 1.20

                    weighted_results.append({'score': s, 'prob': prob, 'm': matches})

                if not weighted_results: continue
                weighted_results.sort(key=lambda x: x['prob'], reverse=True)

                top = weighted_results[0]
                if top['score'] == '1:1' and top['prob'] < 15 and len(weighted_results) > 1:
                    top = weighted_results[1]

                print(f"\n⚽ {f['teams']['home']['name']} - {f['teams']['away']['name']}")
                print(f"📊 Odds: [{h_o} - {a_o}] | Dens(3.5): {dens}")
                print(f"🎯 Choice: {top['score']} ({top['prob']:.1f}% | {top['m']} sims) | {label}")
                print("-" * 40)
            except: continue

if __name__ == "__main__":
    run_pkm_5_0()

--- 🛰 PKM 5.0 STRICT | 08/01 05:19 ---

⚽ Lausanne - Neuchatel Xamax FC
📊 Odds: [1.44 - 5.0] | Dens(3.5): 0.1
🎯 Choice: 1:1 (15.3% | 299 sims) | ⚠️ ЛОВУШКА
----------------------------------------

⚽ Australia U23 - Thailand U23
📊 Odds: [1.22 - 9.4] | Dens(3.5): 0.4651
🎯 Choice: 2:0 (15.5% | 92 sims) | 🔍 СТАНДАРТ
----------------------------------------

⚽ Haras El Hodood - Smouha SC
📊 Odds: [4.0 - 2.0] | Dens(3.5): 0.2105
🎯 Choice: 1:2 (10.3% | 564 sims) | 🔍 СТАНДАРТ
----------------------------------------

⚽ Al-Jazira - Ajman
📊 Odds: [1.52 - 5.5] | Dens(3.5): 0.3636
🎯 Choice: 1:0 (13.2% | 590 sims) | ⚠️ ЛОВУШКА
----------------------------------------

⚽ Dibba Al-Fujairah - Khorfakkan
📊 Odds: [3.05 - 2.08] | Dens(3.5): 0.339
🎯 Choice: 1:2 (7.4% | 645 sims) | 🔍 СТАНДАРТ
----------------------------------------

⚽ Oman Club - Bahla
📊 Odds: [2.25 - 3.2] | Dens(3.5): 0.2105
🎯 Choice: 2:1 (8.9% | 1531 sims) | 🔍 СТАНДАРТ
----------------------------------------

⚽ Al Shahaniya - Al-Duhail

In [1]:
import os, json, math, requests
from datetime import datetime, timedelta, timezone

# --- АВТОКОННЕКТОР ---
try:
    from google.colab import drive
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive', force_remount=True)
except: pass

CONFIG = {
    'API_KEY_SPORTS': 'd5f8091f602cb270431726b11d7100cc',
    'POSSIBLE_PATHS': ['/content/drive/MyDrive/ШКОЛА/patterns.json', '/content/drive/My Drive/ШКОЛА/patterns.json']
}

def mathematician_check(h_o, a_o, target_score, math_db):
    radius = 0.30
    sims = [p['score'] for p in math_db if math.sqrt((p['h']-h_o)**2 + (p['a']-a_o)**2) < radius]
    if not sims: return 0.0, 0
    count = sims.count(target_score)
    return (count / len(sims)) * 100, len(sims)

def get_real_dens(bets):
    """Гибридный Dens: Приоритет 3.5, откат на 2.5 с коэф 0.7"""
    for b in bets:
        if "Over/Under" in b['name']:
            for v in b['values']:
                if v['value'] == 'Over 3.5': return round(1 / float(v['odd']), 4), True
    for b in bets:
        if "Over/Under" in b['name']:
            for v in b['values']:
                if v['value'] == 'Over 2.5': return round((1 / float(v['odd'])) * 0.70, 4), False
    return None, None

def get_recommendation(label, top_score, dens, h_o, a_o):
    """Логика ставок ШКОЛА"""
    home_g, away_g = map(int, top_score.split(':'))
    total = home_g + away_g

    if label == "⚠️ ЛОВУШКА": return "X2 или Ф(+1.5)"
    if label == "🔥 ЖИР": return "ТБ(3) или ОЗ+ТБ(2.5)"
    if label == "💎 БЕТОН (ОЗ)": return "ОЗ (Да)"
    if total >= 3: return "ТБ(2.5)"
    if h_o < 1.50: return "П1 и ТМ(4.5)"
    if a_o < 1.50: return "П2 и ТМ(4.5)"
    return "ИТБ1(1) или ИТБ2(1)"

def run_pkm_5_0():
    path_db = next((p for p in CONFIG['POSSIBLE_PATHS'] if os.path.exists(p)), None)
    if not path_db: return print("❌ База ШКОЛА не найдена")

    with open(path_db, 'r') as f: math_db = json.load(f)
    now = datetime.now(timezone.utc)
    headers = {'x-apisports-key': CONFIG['API_KEY_SPORTS']}
    print(f"--- 🛰 PKM 5.0 BOSS MODE | {now.strftime('%d/%m %H:%M')} ---")

    res = requests.get(f"https://v3.football.api-sports.io/fixtures?date={now.strftime('%Y-%m-%d')}", headers=headers).json()

    for f in res.get('response', []):
        f_dt = datetime.fromisoformat(f['fixture']['date'].replace('Z', '+00:00'))
        if now <= f_dt <= now + timedelta(hours=12):
            try:
                fid = f['fixture']['id']
                o_res = requests.get(f"https://v3.football.api-sports.io/odds?fixture={fid}", headers=headers).json()
                if not o_res['response']: continue

                bookie = o_res['response'][0]['bookmakers'][0]
                odds = {v['value']: float(v['odd']) for b in bookie['bets'] if b['name'] in ['Match Winner', 'Full Time Result'] for v in b['values']}
                h_o, a_o = odds.get('Home', odds.get('1')), odds.get('Away', odds.get('2'))
                if not h_o or not a_o: continue

                dens, is_exact = get_real_dens(bookie['bets'])
                if dens is None: continue

                # Классификация сценария
                is_mirrored = abs(h_o - a_o) < 0.5
                gap = round(abs(h_o - a_o), 2)

                # Физик решает
                if min(h_o, a_o) < 1.60 and dens < 0.45 and not is_mirrored:
                    label, s_list = "⚠️ ЛОВУШКА", (['1:1', '2:1'] if h_o < a_o else ['1:1', '1:2'])
                elif dens > 0.55:
                    label, s_list = "🔥 ЖИР", (['3:1', '4:1'] if h_o < a_o else ['1:3', '1:4'])
                elif is_mirrored and dens > 0.20:
                    label, s_list = "💎 БЕТОН (ОЗ)", ['1:1', '2:1', '1:2', '2:2']
                else:
                    label, s_list = "🔍 СТАНДАРТ", (['2:0', '3:0'] if h_o < a_o else ['0:2', '0:3'])

                # Математик фильтрует
                weighted = []
                for s in s_list:
                    prob, m = mathematician_check(h_o, a_o, s, math_db)
                    if prob == 0: continue
                    # Веса
                    if dens > 0.40 and sum(int(x) for x in s.split(':')) > 2: prob *= 1.15
                    if min(h_o, a_o) < 1.70 and (':0' in s or '0:' in s): prob *= 1.10
                    weighted.append({'score': s, 'prob': prob})

                if not weighted: continue
                weighted.sort(key=lambda x: x['prob'], reverse=True)
                top = weighted[0]

                # Исключение слабого 1:1
                if top['score'] == '1:1' and top['prob'] < 15 and len(weighted) > 1: top = weighted[1]

                # РЕКОМЕНДАЦИЯ
                bet = get_recommendation(label, top['score'], dens, h_o, a_o)

                # ВЫВОД (Scannable)
                print(f"🏆 {f['league']['country'].upper()}: {f['league']['name']}")
                print(f"⏰ {f_dt.strftime('%H:%M')} | {f['teams']['home']['name']} vs {f['teams']['away']['name']}")
                print(f"📊 Линия: {h_o} - {a_o} | Gap: {gap} | Dens: {dens} {'(EXACT)' if is_exact else '(APPROX)'}")
                print(f"🎯 СЧЕТ: {top['score']} | СТАВКА: **{bet}**")
                print(f"📝 Статус: {label}")
                print("-" * 50)

            except: continue

if __name__ == "__main__":
    run_pkm_5_0()

Mounted at /content/drive
--- 🛰 PKM 5.0 BOSS MODE | 12/01 13:36 ---
🏆 WORLD: Friendlies Clubs
⏰ 14:00 | Rapid Vienna vs Ferencvarosi TC
📊 Линия: 2.62 - 2.3 | Gap: 0.32 | Dens: 0.4242 (APPROX)
🎯 СЧЕТ: 1:1 | СТАВКА: **ОЗ (Да)**
📝 Статус: 💎 БЕТОН (ОЗ)
--------------------------------------------------
🏆 BRAZIL: São Paulo Youth Cup
⏰ 14:00 | Cruzeiro U20 vs EC Meia Noite U20
📊 Линия: 1.14 - 17.0 | Gap: 15.86 | Dens: 0.3636 (EXACT)
🎯 СЧЕТ: 2:1 | СТАВКА: **X2 или Ф(+1.5)**
📝 Статус: ⚠️ ЛОВУШКА
--------------------------------------------------
🏆 EGYPT: League Cup
⏰ 15:00 | Enppi vs El Mokawloon
📊 Линия: 2.0 - 4.0 | Gap: 2.0 | Dens: 0.2105 (EXACT)
🎯 СЧЕТ: 2:0 | СТАВКА: **ИТБ1(1) или ИТБ2(1)**
📝 Статус: 🔍 СТАНДАРТ
--------------------------------------------------
🏆 EGYPT: League Cup
⏰ 15:00 | Ghazl El Mehalla vs Ceramica Cleopatra
📊 Линия: 4.33 - 1.91 | Gap: 2.42 | Dens: 0.2692 (APPROX)
🎯 СЧЕТ: 0:2 | СТАВКА: **ИТБ1(1) или ИТБ2(1)**
📝 Статус: 🔍 СТАНДАРТ
----------------------------------------